In [0]:
import pandas as pd
import numpy as np
from collections import Counter

In [0]:
raw = pd.read_csv("data/train/train.csv")
campaign_data = np.load("npy/campaign_data.npy", allow_pickle=True)
coup_item_map = pd.read_csv("data/train/coupon_item_mapping.csv")
cust_trans = np.load("npy/customer_transaction_data.npy", allow_pickle=True)
cust_demographics = np.load("npy/customer_demographics.npy", allow_pickle=True)
item_data = np.load("npy/item_data.npy", allow_pickle=True)
print("Shape = ", raw.shape)
raw.head()

Shape =  (78369, 5)


,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [0]:
raw_campaign = raw.merge(campaign_data, how='left').drop(['campaign_id'],axis=1)
coup_item_data = coup_item_map.merge(item_data, how='left').drop(['item_id'], axis=1).drop_duplicates()
raw_campaign_coupItem = raw_campaign.merge(coup_item_data, how="left")
raw_campaign_coupItem_cust = raw_campaign_coupItem.merge(cust_demographics, how="left")
raw_campaign_coupItem_cust.fillna(-1,inplace=True)
print("Shape = ", raw_campaign_coupItem_cust.shape)
raw_campaign_coupItem_cust.head()

Shape =  (580471, 59)


,id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,start_day,start_month,start_year,...,enc_marital_status,Age_18-25,Age_26-35,Age_36-45,Age_46-55,Age_56-70,Age_70+,Married,Single,Unknown
0,1,27,1053,0,X,2013-05-19,2013-07-05,19,5,13,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,27,1053,0,X,2013-05-19,2013-07-05,19,5,13,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2,116,48,0,X,2013-05-19,2013-07-05,19,5,13,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,6,635,205,0,Y,2013-03-11,2013-04-12,11,3,13,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,7,644,1050,0,X,2013-05-19,2013-07-05,19,5,13,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [0]:
X = raw_campaign_coupItem_cust.drop(['id','coupon_id','customer_id','campaign_type','start_date','end_date','brand_type',
                                     'category', 'age_range','marital_status','redemption_status'], axis=1)
y = raw_campaign_coupItem_cust.redemption_status

print("Shape = ", X.shape)
X.head()

Shape =  (580471, 48)


,start_day,start_month,start_year,end_day,end_month,end_year,enc_campaign_type,campaign_X,campaign_Y,brand,...,enc_marital_status,Age_18-25,Age_26-35,Age_36-45,Age_46-55,Age_56-70,Age_70+,Married,Single,Unknown
0,19,5,13,5,7,13,0,1,0,1636,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,19,5,13,5,7,13,0,1,0,1105,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,19,5,13,5,7,13,0,1,0,56,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,11,3,13,12,4,13,1,0,1,560,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,19,5,13,5,7,13,0,1,0,611,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [0]:
model = RandomForestClassifier(random_state=10, max_depth=30, n_estimators=15, min_samples_split=20, min_samples_leaf=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Score = ", roc_auc_score(y_test, y_pred))
pd.DataFrame([model.feature_importances_], columns=X.columns).transpose()

Score =  0.7017014543841037


,0
start_day,3.756721e-02
start_month,2.063393e-02
start_year,2.078387e-03
end_day,2.717534e-03
end_month,1.762601e-02
end_year,7.107378e-04
enc_campaign_type,2.001647e-03
campaign_X,1.408892e-03
campaign_Y,7.357601e-04
brand,9.044363e-02


In [0]:
model = XGBClassifier(random_state=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Score = ", roc_auc_score(y_test, y_pred))
pd.DataFrame([model.feature_importances_], columns=X.columns).transpose()

Score =  0.5411842468450169


array([0.01931649, 0.04160475, 0.        , 0.03120357, 0.04457652,
       0.        , 0.01337296, 0.        , 0.        , 0.02971768,
       0.01485884, 0.02080238, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.02377415, 0.        , 0.        ,
       0.05200594, 0.00742942, 0.00148588, 0.04011887, 0.        ,
       0.04606241, 0.        , 0.        , 0.        , 0.00297177,
       0.        , 0.        , 0.        , 0.04309064, 0.09658246,
       0.06686478, 0.21693908, 0.0282318 , 0.01485884, 0.01040119,
       0.03417533, 0.01931649, 0.00742942, 0.01783061, 0.        ,
       0.        , 0.05497771, 0.        ], dtype=float32)

In [0]:
raw_campaign_coupItem_cust.to_csv("data.csv")